In [334]:
import pandas as pd
import numpy as np

In [335]:
start_full = pd.read_csv('/Users/sj/Downloads/Test-master/sample_2_pre_100028.csv', encoding="ISO-8859-1", low_memory=False)

In [336]:
start_full['uin'] = start_full.index + 1000
start_full.head(5)

,ZZ_APP_ID,ZZ_CIN_ID,ZZ_ACCT_ID,applic_date,transaction_type,transaction_amount,transaction_narrative,PrimaryCategoryDescription,SubCategoryDescription,VendorDescription,...,Transaction_Description2,MCC_1,MCC_Plus,MCC_AS,MCC_Label,PrimaryCategory_MCC_AS,SubCategory_MCC_AS,Enhance_PrimaryCategory,Enhance_SubCategory,uin
0,10195,100028,10000134,28/11/2016,D/D,-64.00,SE GAS LIMITED,Utilities,Energy Providers,Southern Electric,...,SE GAS LIMITED,NaN,Uncategorized MCC code,Unavailable MCC code,NaN,Unavailable MCC code,NaN,Utilities,Energy Providers,1000
1,10195,100028,10000134,28/11/2016,D/D,-1219.46,HALIFAX,Rent and Mortgage,Banks And Building Societies,Halifax,...,HALIFAX,NaN,Uncategorized MCC code,Unavailable MCC code,NaN,Unavailable MCC code,NaN,Rent and Mortgage,Banks And Building Societies,1001
2,10195,100028,10000134,28/11/2016,D/D,-189.61,SHAWBROOK,Financial Services,Personal Loans,Shawbrook,...,SHAWBROOK,NaN,Uncategorized MCC code,Unavailable MCC code,NaN,Unavailable MCC code,NaN,Financial Services,Personal Loans,1002
3,10195,100028,10000134,28/11/2016,D/D,-64.00,SE GAS LIMITED,Utilities,Energy Providers,Southern Electric,...,SE GAS LIMITED,NaN,Uncategorized MCC code,Unavailable MCC code,NaN,Unavailable MCC code,NaN,Utilities,Energy Providers,1003
4,10195,100028,10000134,28/11/2016,POS -,-22.00,NEXT 0209 BLUEWATER GREENHITHE,Shopping (Fashion),General Fashion,Next,...,NEXT 0209 BLUEWATER GREENHITHE,5651.0,Clothing and Retail,Shopping (Fashion)|General Fashion,FAMILY CLOTHING STORES,Shopping (Fashion),General Fashion,Shopping (Fashion),General Fashion,1004


In [337]:
start = start_full.loc[(start.ZZ_CIN_ID == 100028),:].reset_index(drop=True)

In [338]:
start.shape

(1298, 33)

In [339]:
start.columns

Index(['ZZ_APP_ID', 'ZZ_CIN_ID', 'ZZ_ACCT_ID', 'applic_date',
       'transaction_type', 'transaction_amount', 'transaction_narrative',
       'PrimaryCategoryDescription', 'SubCategoryDescription',
       'VendorDescription', 'RecurrenceEventId', 'RecurrenceType',
       'merchant_MCC', 'TRANSACTION_DATE', 'trans_id_app', 'Segment',
       'ZZ_JOINT', 'CIN_M_OR_J', 'appsubject', 'cancelled_transaction',
       'trans_id_cin', 'card_type', 'Deb_Cred_Flag',
       'Transaction_Description2', 'MCC_1', 'MCC_Plus', 'MCC_AS', 'MCC_Label',
       'PrimaryCategory_MCC_AS', 'SubCategory_MCC_AS',
       'Enhance_PrimaryCategory', 'Enhance_SubCategory', 'uin'],
      dtype='object')

In [340]:
my_columns = ['ZZ_CIN_ID',  'transaction_amount', 'ZZ_ACCT_ID',
       'PrimaryCategoryDescription', 'SubCategoryDescription',
       'TRANSACTION_DATE',  'Deb_Cred_Flag', 'uin']

In [341]:
start = start.loc[:,my_columns]

In [342]:
start.shape

(1298, 8)

In [343]:
start["trn_count"] = start.groupby(['ZZ_CIN_ID', 'TRANSACTION_DATE','transaction_amount' ])['ZZ_CIN_ID'].transform("count")

In [344]:
deb_idx =( (start['trn_count'] == 1) & (
    (start['PrimaryCategoryDescription'] == 'Transfers / Other') | 
    (start['PrimaryCategoryDescription'] == 'Misc Regular Payments')) &
         (start["Deb_Cred_Flag"] == "Debit"))

In [345]:
cr_idx =( (start['trn_count'] == 1) & (
    (start['PrimaryCategoryDescription'] == 'Transfers / Other') | 
    (start['PrimaryCategoryDescription'] == 'Misc Regular Payments')) &
         (start["Deb_Cred_Flag"] == "Credit"))

In [346]:
df_debit = start.loc[deb_idx, :]#['ZZ_CIN_ID', 'TRANSACTION_DATE','transaction_amount']]
df_debit.shape

(107, 9)

In [347]:
df_credit = start.loc[cr_idx, :]#['ZZ_CIN_ID', 'TRANSACTION_DATE','transaction_amount']]
df_credit.shape

(124, 9)

In [348]:
df_debit["transaction_amount_key"] = df_debit.loc[:, "transaction_amount"] * -1
df_credit["transaction_amount_key"] = df_credit.loc[:, "transaction_amount"] 

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [349]:
keys = ['ZZ_CIN_ID', 'TRANSACTION_DATE', "transaction_amount_key"]
keys1 = ['ZZ_CIN_ID', 'ZZ_ACCT_ID', 'TRANSACTION_DATE', "transaction_amount_key"]
same_df = pd.merge(df_credit, df_debit, left_on=keys1, right_on=keys1, how="inner")

In [350]:
same_df.shape

(3, 16)

In [351]:
same_df.columns

Index(['ZZ_CIN_ID', 'transaction_amount_x', 'ZZ_ACCT_ID',
       'PrimaryCategoryDescription_x', 'SubCategoryDescription_x',
       'TRANSACTION_DATE', 'Deb_Cred_Flag_x', 'uin_x', 'trn_count_x',
       'transaction_amount_key', 'transaction_amount_y',
       'PrimaryCategoryDescription_y', 'SubCategoryDescription_y',
       'Deb_Cred_Flag_y', 'uin_y', 'trn_count_y'],
      dtype='object')

In [352]:
d = same_df[keys1]

In [353]:
d

,ZZ_CIN_ID,ZZ_ACCT_ID,TRANSACTION_DATE,transaction_amount_key
0,100028,10011530,11/03/2016,10000.0
1,100028,10011530,10/03/2016,10000.0
2,100028,10011530,28/07/2016,450.0


In [354]:
d['dup'] = 1
cr_df = pd.merge(df_credit, d, left_on=keys1, right_on=keys1, how="left")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [355]:
dd = cr_df.loc[(cr_df.dup != 1)]

In [356]:
db_df = pd.merge(df_debit, d, left_on=keys1, right_on=keys1, how="left")

In [357]:
de = db_df.loc[(db_df.dup != 1)]

In [358]:
k1 = []
res_df = pd.merge(dd, de, left_on=keys, right_on=keys, how="inner")
res_df.shape

(37, 19)

In [359]:
res_df.columns

Index(['ZZ_CIN_ID', 'transaction_amount_x', 'ZZ_ACCT_ID_x',
       'PrimaryCategoryDescription_x', 'SubCategoryDescription_x',
       'TRANSACTION_DATE', 'Deb_Cred_Flag_x', 'uin_x', 'trn_count_x',
       'transaction_amount_key', 'dup_x', 'transaction_amount_y',
       'ZZ_ACCT_ID_y', 'PrimaryCategoryDescription_y',
       'SubCategoryDescription_y', 'Deb_Cred_Flag_y', 'uin_y', 'trn_count_y',
       'dup_y'],
      dtype='object')

In [360]:
res_df = res_df.loc[res_df.transaction_amount_x > 0, :]

In [361]:
res_df.shape

(37, 19)

In [362]:
res_df.columns

Index(['ZZ_CIN_ID', 'transaction_amount_x', 'ZZ_ACCT_ID_x',
       'PrimaryCategoryDescription_x', 'SubCategoryDescription_x',
       'TRANSACTION_DATE', 'Deb_Cred_Flag_x', 'uin_x', 'trn_count_x',
       'transaction_amount_key', 'dup_x', 'transaction_amount_y',
       'ZZ_ACCT_ID_y', 'PrimaryCategoryDescription_y',
       'SubCategoryDescription_y', 'Deb_Cred_Flag_y', 'uin_y', 'trn_count_y',
       'dup_y'],
      dtype='object')

In [363]:
res_df["flag"] = "own_trn"

In [364]:
keep_colsx = ["flag","TRANSACTION_DATE",
             "transaction_amount_x","ZZ_CIN_ID","SubCategoryDescription_x",
             "PrimaryCategoryDescription_x","Deb_Cred_Flag_x"]
keep_colsy = ["flag","TRANSACTION_DATE","transaction_amount_y",
              "ZZ_CIN_ID","SubCategoryDescription_y",
              "PrimaryCategoryDescription_y","Deb_Cred_Flag_y"]

In [365]:
start2 = pd.merge(start, 
         res_df[keep_colsx], 
         right_on=["TRANSACTION_DATE", "transaction_amount_x","ZZ_CIN_ID","SubCategoryDescription_x","PrimaryCategoryDescription_x","Deb_Cred_Flag_x"], 
         left_on=["TRANSACTION_DATE", "transaction_amount","ZZ_CIN_ID","SubCategoryDescription","PrimaryCategoryDescription","Deb_Cred_Flag"], 
         how='outer')

In [366]:
start2.shape

(1298, 14)

In [367]:
start2 = pd.merge(start2, 
         res_df[keep_colsy], 
         right_on=["TRANSACTION_DATE", "transaction_amount_y","ZZ_CIN_ID","SubCategoryDescription_y","PrimaryCategoryDescription_y","Deb_Cred_Flag_y"], 
         left_on=["TRANSACTION_DATE", "transaction_amount","ZZ_CIN_ID","SubCategoryDescription","PrimaryCategoryDescription","Deb_Cred_Flag"], 
         how='left')

In [368]:
start2.shape

(1298, 19)

In [369]:
start2.columns

Index(['ZZ_CIN_ID', 'transaction_amount', 'ZZ_ACCT_ID',
       'PrimaryCategoryDescription', 'SubCategoryDescription',
       'TRANSACTION_DATE', 'Deb_Cred_Flag', 'uin', 'trn_count', 'flag_x',
       'transaction_amount_x', 'SubCategoryDescription_x',
       'PrimaryCategoryDescription_x', 'Deb_Cred_Flag_x', 'flag_y',
       'transaction_amount_y', 'SubCategoryDescription_y',
       'PrimaryCategoryDescription_y', 'Deb_Cred_Flag_y'],
      dtype='object')

In [370]:
start2.tail(5)

,ZZ_CIN_ID,transaction_amount,ZZ_ACCT_ID,PrimaryCategoryDescription,SubCategoryDescription,TRANSACTION_DATE,Deb_Cred_Flag,uin,trn_count,flag_x,transaction_amount_x,SubCategoryDescription_x,PrimaryCategoryDescription_x,Deb_Cred_Flag_x,flag_y,transaction_amount_y,SubCategoryDescription_y,PrimaryCategoryDescription_y,Deb_Cred_Flag_y
1293,100028,0.00,10011530,Commuting and travel,Trains and coaches,31/10/2016,Debit,2293,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1294,100028,158.76,10011530,Refund,POS Credit,17/10/2016,Credit,2294,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1295,100028,-10.99,10011530,Shopping (Discretionary),Music and Downloads,24/10/2016,Debit,2295,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1296,100028,-10.00,10011530,Shopping (Discretionary),E-Commerce Payment System,27/10/2016,Debit,2296,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,100028,-9.30,10011530,Misc Card Spend,NaN,13/10/2016,Debit,2297,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [371]:
start2.loc[(start2.flag_x == 'own_trn')].shape

(37, 19)

In [372]:
start2.loc[(start2.flag_y == 'own_trn')].shape

(37, 19)

In [373]:
start2.loc[start2["flag_x"].isna(), "flag_x"] = start2.loc[start2["flag_x"].isna(), "flag_y"]

In [374]:
start2["flag_x"].value_counts()

own_trn    74
Name: flag_x, dtype: int64

In [375]:
start2.shape

(1298, 19)

In [376]:
start2.columns

Index(['ZZ_CIN_ID', 'transaction_amount', 'ZZ_ACCT_ID',
       'PrimaryCategoryDescription', 'SubCategoryDescription',
       'TRANSACTION_DATE', 'Deb_Cred_Flag', 'uin', 'trn_count', 'flag_x',
       'transaction_amount_x', 'SubCategoryDescription_x',
       'PrimaryCategoryDescription_x', 'Deb_Cred_Flag_x', 'flag_y',
       'transaction_amount_y', 'SubCategoryDescription_y',
       'PrimaryCategoryDescription_y', 'Deb_Cred_Flag_y'],
      dtype='object')

In [377]:
sj = ['uin', 'flag_x']
s = start2[sj]

In [378]:
s.columns

Index(['uin', 'flag_x'], dtype='object')

In [379]:
s.shape

(1298, 2)

In [380]:
final_df = pd.merge(start_full, s, left_on='uin', right_on='uin', how='inner')

In [381]:
final_df.shape

(1298, 34)

In [382]:
final_df.columns

Index(['ZZ_APP_ID', 'ZZ_CIN_ID', 'ZZ_ACCT_ID', 'applic_date',
       'transaction_type', 'transaction_amount', 'transaction_narrative',
       'PrimaryCategoryDescription', 'SubCategoryDescription',
       'VendorDescription', 'RecurrenceEventId', 'RecurrenceType',
       'merchant_MCC', 'TRANSACTION_DATE', 'trans_id_app', 'Segment',
       'ZZ_JOINT', 'CIN_M_OR_J', 'appsubject', 'cancelled_transaction',
       'trans_id_cin', 'card_type', 'Deb_Cred_Flag',
       'Transaction_Description2', 'MCC_1', 'MCC_Plus', 'MCC_AS', 'MCC_Label',
       'PrimaryCategory_MCC_AS', 'SubCategory_MCC_AS',
       'Enhance_PrimaryCategory', 'Enhance_SubCategory', 'uin', 'flag_x'],
      dtype='object')

In [383]:
final_df.PrimaryCategoryDescription.value_counts()

Transfers / Other                  198
Shopping (Discretionary)           165
Misc Card Spend                    115
Utilities                          101
Cash Machine                        99
Financial Services                  93
Amazon All                          63
Insurance                           60
Gambling and Betting                59
Bank Charges and Returns            59
Shopping (Household Essentials)     45
Commuting and travel                41
Misc Regular Payments               37
Shopping (Fashion)                  23
Restaurants and takeaway            23
Automotive                          21
Rent and Mortgage                   19
Interest                            17
Services                            15
Identified Salary                   10
Entertainment                        9
Cash Deposit                         7
Refund                               7
Personal Healthcare                  4
Shopping (Home)                      3
Books, Newspapers and Mag

In [385]:
final_df.loc[final_df.flag_x == 'own_trn', 'PrimaryCategoryDescription'] = 'OWN TRANSFERS'

In [386]:
final_df.PrimaryCategoryDescription.value_counts()

Shopping (Discretionary)           165
Transfers / Other                  136
Misc Card Spend                    115
Utilities                          101
Cash Machine                        99
Financial Services                  93
OWN TRANSFERS                       74
Amazon All                          63
Insurance                           60
Bank Charges and Returns            59
Gambling and Betting                59
Shopping (Household Essentials)     45
Commuting and travel                41
Misc Regular Payments               25
Restaurants and takeaway            23
Shopping (Fashion)                  23
Automotive                          21
Rent and Mortgage                   19
Interest                            17
Services                            15
Identified Salary                   10
Entertainment                        9
Refund                               7
Cash Deposit                         7
Personal Healthcare                  4
Shopping (Home)          

In [51]:
# cnt = start2.loc[(start2.flag_x == 'own_trn')]

In [19]:
#count = 1 and the transaction type = “DEBIT” and the Primary Category is either Transfers / Other r Misc Regular Payments
deb_idx =( (df_debit['trn_count'] == 1) & (
    (df_debit['PrimaryCategoryDescription'] == 'Transfers / Other') | 
    (df_debit['PrimaryCategoryDescription'] == 'Misc Regular Payments'))&
         (df_debit["Deb_Cred_Flag"] == "Debit"))

df["deb_amt"] = 0
df.loc[deb_idx,"deb_amt"] = df.loc[deb_idx,"transaction_amount"]


In [20]:
#count = 1 and the transaction type = “DEBIT” and the Primary Category is either Transfers / Other r Misc Regular Payments
cr_idx =( (df['trn_count'] == 1) & (
    (df['PrimaryCategoryDescription'] == 'Transfers / Other') | 
    (df['PrimaryCategoryDescription'] == 'Misc Regular Payments'))&
         (df["Deb_Cred_Flag"] == "Credit"))

df["cr_amt"] = 0
df.loc[cr_idx,"cr_amt"] = df.loc[cr_idx,"transaction_amount"]

In [22]:
df['deb_sum'] = df.groupby(['ZZ_CIN_ID', 'TRANSACTION_DATE','transaction_amount' ])['deb_amt'].transform("sum")
df['cr_sum'] = df.groupby(['ZZ_CIN_ID', 'TRANSACTION_DATE','transaction_amount' ])['cr_amt'].transform("sum")

In [24]:
df.sort_values(by=['ZZ_CIN_ID', 'TRANSACTION_DATE','transaction_amount' ])

,ZZ_CIN_ID,transaction_amount,PrimaryCategoryDescription,SubCategoryDescription,TRANSACTION_DATE,Deb_Cred_Flag,trn_count,deb_amt,cr_amt,deb_sum,cr_sum
364,100008,-16.95,Misc Card Spend,NaN,01/01/2016,Debit,1,0.00,0.00,0.00,0.00
103,100008,-14.66,Shopping (Household Essentials),General Groceries,01/02/2016,Debit,1,0.00,0.00,0.00,0.00
184,100008,100.00,Cash Deposit,NaN,01/02/2016,Credit,1,0.00,0.00,0.00,0.00
267,100008,-20.99,Transfers / Other,Bank Transfer,01/04/2016,Debit,1,-20.99,0.00,-20.99,0.00
591,100008,-20.00,Transfers / Other,Bank Transfer,01/04/2016,Debit,1,-20.00,0.00,-20.00,0.00
68,100008,20.00,Transfers / Other,Bank Transfer,01/04/2016,Credit,1,0.00,20.00,0.00,20.00
532,100008,20.99,Transfers / Other,Bank Transfer,01/04/2016,Credit,1,0.00,20.99,0.00,20.99
549,100008,-300.00,Savings,NaN,01/06/2016,Debit,1,0.00,0.00,0.00,0.00
511,100008,-200.00,Savings,NaN,01/06/2016,Debit,1,0.00,0.00,0.00,0.00
83,100008,-19.01,Transfers / Other,Bank Transfer,01/06/2016,Debit,1,-19.01,0.00,-19.01,0.00


In [259]:
# df.head(2)

In [260]:
ot_df = df.loc[(df.PrimaryCategoryDescription == 'Transfers / Other') | (df.PrimaryCategoryDescription == 'Misc Regular Payments')]

In [261]:
ot_df.shape

(159005, 32)

In [262]:
cd_df = ot_df.loc[(df.Deb_Cred_Flag == 'Credit') | (df.Deb_Cred_Flag == 'Debit')]

In [263]:
cd_df.shape

(159005, 32)

In [264]:
df_freq = cd_df.groupby(['ZZ_CIN_ID', 'transaction_amount', 'TRANSACTION_DATE', 'PrimaryCategoryDescription', 'SubCategoryDescription'])['transaction_amount'].count()

In [265]:
df_freq.shape

(97715,)

In [266]:
df_uni = df_freq[df_freq == 1]
df_uni.shape

(93890,)

In [267]:
df_freq2 = pd.DataFrame(df_freq)
df_freq2['amt_count'] = df_freq2['transaction_amount']
df_freq2.drop(columns=['transaction_amount'], inplace=True)
df_freq2 = df_freq2.reset_index()

In [268]:
df_freq2.shape

(97715, 6)

In [269]:
df_freq2['amt_sum'] = df_freq2.groupby(['ZZ_CIN_ID', 'TRANSACTION_DATE', 'PrimaryCategoryDescription', 'SubCategoryDescription'])['transaction_amount'].transform('sum')

In [270]:
df_freq2.head(5)

,ZZ_CIN_ID,transaction_amount,TRANSACTION_DATE,PrimaryCategoryDescription,SubCategoryDescription,amt_count,amt_sum
0,100008,-3000.00,08/04/2016,Transfers / Other,Bank Transfer,1,-4.200000e+03
1,100008,-2232.20,25/11/2015,Transfers / Other,Bank Transfer,1,0.000000e+00
2,100008,-1200.00,08/04/2016,Transfers / Other,Bank Transfer,1,-4.200000e+03
3,100008,-1116.10,25/11/2015,Transfers / Other,Bank Transfer,1,0.000000e+00
4,100008,-1011.32,24/05/2016,Transfers / Other,Bank Transfer,1,-1.136868e-13


In [271]:
df_freq2.shape

(97715, 7)

In [272]:
idx = (df_freq2.amt_sum == 0)

In [273]:
df_freq2["flag"]= np.nan
df_freq2.loc[idx, "flag"] = "OWN TRANSFER"
df_freq2.shape

(97715, 8)

In [277]:
res_df = df_freq2.loc[(df_freq2.flag == 'OWN TRANSFER')]
# 34,256 
res_df.shape

(21692, 8)

In [278]:
res = pd.merge(df, df_freq2, left_on=['ZZ_CIN_ID', 'TRANSACTION_DATE', 'transaction_amount', 'PrimaryCategoryDescription', 'SubCategoryDescription'], right_on=['ZZ_CIN_ID', 'TRANSACTION_DATE', 'transaction_amount', 'PrimaryCategoryDescription', 'SubCategoryDescription'], how='left')

In [279]:
res.loc[(res.ZZ_CIN_ID == 100008) & (res.flag == 'OWN TRANSFER'), :].shape

(176, 35)

In [246]:
cus1 = res.loc[(res.ZZ_CIN_ID == 100008),['ZZ_CIN_ID', 'TRANSACTION_DATE', 'transaction_amount',"flag"] ]
cus1.head()

,ZZ_CIN_ID,TRANSACTION_DATE,transaction_amount,flag
436815,100008,14/01/2016,-3.96,NaN
436816,100008,25/01/2016,-100.50,NaN
436817,100008,06/02/2016,-5.78,NaN
436818,100008,18/02/2016,30.00,NaN
436819,100008,04/11/2015,20.00,OWN TRANSFER


In [247]:
cus1.shape

(594, 4)

In [248]:
cus1.dtypes

ZZ_CIN_ID               int64
TRANSACTION_DATE       object
transaction_amount    float64
flag                   object
dtype: object

In [255]:
cus1.groupby(['ZZ_CIN_ID',"TRANSACTION_DATE",  'transaction_amount']).count()

flag
ZZ_CIN_ID TRANSACTION_DATE transaction_amount      
100008    01/01/2016       -16.95                 0
          01/02/2016       -14.66                 0
                            100.00                0
          01/04/2016       -20.99                 0
                           -20.00                 0
                            20.00                 0
                            20.99                 0
          01/06/2016       -300.00                0
                           -200.00                0
                           -19.01                 1
                            19.01                 1
          01/07/2016       -300.00                0
                           -200.00                0
          01/08/2016       -300.00                0
                           -200.00                0
                           -35.78                 0
          01/09/2016       -300.00                0
                           -200.00                0
          01/10/2016       -4.49                  0
          02/02/2016       -20.00                 0
          02/05/2016       -22.00                 0
                           -10.00                 0
          02/06/2016       -30.00                 1
                           -5.50                  0
                            30.00                 1
          02/09/2016       -83.65                 0
                           -5.00                  0
                            150.00                0
          02/10/2016       -1.58                  0
                            0.00                  0
...                                             ...
          29/10/2016       -12.97                 0
          29/11/2015       -31.98                 0
          29/12/2015       -87.00                 0
                           -50.00                 1
                            50.00                 1
          30/03/2016       -1000.00               1
                           -917.99                1
                           -109.49                0
                            917.99                1
                            1000.00               1
          30/05/2016       -74.70                 0
                           -7.80                  0
          30/11/2015       -66.00                 1
                           -40.00                 1
                           -38.37                 1
                            38.37                 1
                            40.00                 1
                            66.00                 1
          31/05/2016       -50.00                 0
                           -30.00                 0
                           -2.69                  1
                           -0.99                  0
                            2.69                  1
                            50.00                 0
          31/10/2016       -60.00                 0
                           -14.09                 0
                            30.00                 0
                            60.00                 0
          31/12/2015       -30.78                 1
                            30.78                 1

[577 rows x 1 columns]

In [249]:
# read one cust
cus1g = pd.read_csv("C:/Users/sxj335/PycharmProjects/Pytest/data/sample_2_post_100008.csv")

In [250]:
cus1g.shape

(594, 4)

In [251]:
cus1g.groupby(['ZZ_CIN_ID',"TRANSACTION_DATE",  'transaction_amount']).count()

Tran_Date
ZZ_CIN_ID TRANSACTION_DATE transaction_amount           
100008    01/01/2016       -16.95                      1
          01/02/2016       -14.66                      1
                            100.00                     1
          01/04/2016       -20.99                      1
                           -20.00                      1
                            20.00                      1
                            20.99                      1
          01/06/2016       -300.00                     1
                           -200.00                     1
                           -19.01                      1
                            19.01                      1
          01/07/2016       -300.00                     1
                           -200.00                     1
          01/08/2016       -300.00                     1
                           -200.00                     1
                           -35.78                      1
          01/09/2016       -300.00                     1
                           -200.00                     1
          01/10/2016       -4.49                       1
          02/02/2016       -20.00                      1
          02/05/2016       -22.00                      1
                           -10.00                      1
          02/06/2016       -30.00                      1
                           -5.50                       1
                            30.00                      1
          02/09/2016       -83.65                      1
                           -5.00                       1
                            150.00                     1
          02/10/2016       -1.58                       1
                            0.00                       1
...                                                  ...
          29/10/2016       -12.97                      1
          29/11/2015       -31.98                      1
          29/12/2015       -87.00                      1
                           -50.00                      1
                            50.00                      1
          30/03/2016       -1000.00                    1
                           -917.99                     1
                           -109.49                     1
                            917.99                     1
                            1000.00                    1
          30/05/2016       -74.70                      1
                           -7.80                       1
          30/11/2015       -66.00                      1
                           -40.00                      1
                           -38.37                      1
                            38.37                      1
                            40.00                      1
                            66.00                      1
          31/05/2016       -50.00                      2
                           -30.00                      1
                           -2.69                       1
                           -0.99                       1
                            2.69                       1
                            50.00                      2
          31/10/2016       -60.00                      1
                           -14.09                      1
                            30.00                      1
                            60.00                      1
          31/12/2015       -30.78                      1
                            30.78                      1

[577 rows x 1 columns]

In [252]:
cus1g.head()

,ZZ_CIN_ID,transaction_amount,Tran_Date,TRANSACTION_DATE
0,100008,-20.00,22/01/2016,22/01/2016
1,100008,-2.69,31/05/2016,31/05/2016
2,100008,-34.99,27/03/2016,27/03/2016
3,100008,-30.00,13/09/2016,13/09/2016
4,100008,-3.13,14/04/2016,14/04/2016


In [253]:
cus1g.dtypes

ZZ_CIN_ID               int64
transaction_amount    float64
Tran_Date              object
TRANSACTION_DATE       object
dtype: object

In [254]:
pd.merge(cus1, cus1g, left_on=['ZZ_CIN_ID',"TRANSACTION_DATE",  'transaction_amount'], right_on=['ZZ_CIN_ID',"TRANSACTION_DATE", 'transaction_amount'], how='left').\
sort_values(by=['ZZ_CIN_ID',"TRANSACTION_DATE",  'transaction_amount'])

,ZZ_CIN_ID,TRANSACTION_DATE,transaction_amount,flag,Tran_Date
380,100008,01/01/2016,-16.95,NaN,01/01/2016
106,100008,01/02/2016,-14.66,NaN,01/02/2016
191,100008,01/02/2016,100.00,NaN,01/02/2016
278,100008,01/04/2016,-20.99,NaN,01/04/2016
625,100008,01/04/2016,-20.00,NaN,01/04/2016
69,100008,01/04/2016,20.00,NaN,01/04/2016
562,100008,01/04/2016,20.99,NaN,01/04/2016
579,100008,01/06/2016,-300.00,NaN,01/06/2016
540,100008,01/06/2016,-200.00,NaN,01/06/2016
85,100008,01/06/2016,-19.01,OWN TRANSFER,01/06/2016
